# 1 Sentiment Analysis of Tweets

## Проект по Обработка на природните јазици 2016

Изработено од:
Симона Андриеска 121143 и
Зојче Атанасова 125041

## Задача
За дадено множество на твитови да се одреди емоцијата содржана во текстот. Се одредува само дали твитот припаѓа на позитивна или негативна класа според емоцијата во текстот.

## Податочно множество
Податочното множество е земено од натпреварот SemEval 2016. SemEval (Semantic Evaluation) е серија од натпревари каде се евалуираат компјутерски системи за семантичка анализа. Натпреварите се поделени во неколку групи и секоја група има дефинирано по неколку задачи. Втората група (Sentiment Analysis) има четири задачи, од кои првата задача е Sentiment Analysis in Twitter. Оваа задача е најпопуларна, 4 години по ред има најмногу поднесени решенија. Во задачата се дефинирани неколку подзадачи, првата од тие задачи се нарекува Message Polarity Classification, дефинирана е како: Ако е зададен твит, предвидете дали емоцијата/чувството изразено е позитивно, негативно или неутрално.

За целите на натпреварот дадени се податочни множества за тренирање и тестирање. Податочните множества се опишани во [1] а податоците и инструкции како може да се добијат има на [2][3][4]

Во директориумот каде што се наоѓа оваа датотека (sentiment_analysis_of_tweets.ipynb) треба да постои поддиректориум `train_and_dev_data` и во него да има директориуми `original` и  `downloaded_tweets`. 

Податоците се поделени во 5 множества: 
* Dev - за тренирање додека се развива решението
* Dev-Test - за тестирање додека се развива решението
* Train - за тренирање кога се прави евалуација на решението
* Test - за тестирање кога се прави евалуација на решението
* Test-Solution - за проверка на класификацијата

`original` директориумот ги содржи датотеките дадени од семевал
* 100_topics_100_twets.sentence-three-point.subtask-A.dev.gold.txt
 * Содржи 2 колони tweet_id, класа, претставува мало множество кое треба да се користи при развој на решението, како тренирачко множество
* 100_topics_100_tweets.sentence-three-point.subtask-A.devtest.gold.txt
 * Содржи 2 колони tweet_id, класа, претставува мало множество кое треба да се користи при развој на решението, како множество за тестирање
* 100_topics_100_tweets.sentence-three-point.subtask-A.train.gold.txt
 * Содржи 2 колони, tweet_id и класа, множество кое треба да се користи за тренирање
* SemEval2016_task_4_test_gold.txt
 * Содржи 3 колони, реден број на твитот, категорија, класа на која навистина припаѓа, треба да се користи за евалуација на решенијата
* SemEval2016-task-4-test.subtask-A.txt
 * Содржи 3 колони, реден број на твитот, непозната категорија и Текст на твитот, треба да се користи како влез во нашето решение при фазата на тест
 
По кратко процесирање на дадените датотеки од натпреварувањето датотеките за тренирање и тестирање ги доведовме во tab separated формат каде секоја редица претставува еден твит. Во датотеките за тренирање има 2 колони, текст на твитот и sentiment класа на која припаѓа. Во датотеките за тестирање има исто 2 колони, реден број на твитот и текст на твитот. Test-Solution датотеката содржи 2 колони, реден број на твитот (кој се тестирал) и точната класа на која треба да припаѓа. 

Резултат од алгоритмот треба да е една tsv датотека со 2 колони, реден број на твитот и класа која била предвидена од нашиот модел. 

Распределбата на твитови во датотеките дадени од натпреварот е следнава:

||Позитивни|Неутрални|Негативни|*Вкупно*|
|----------|--------:|--------:|-------:|
|Train|3,094|2,043|863|6,000|
|Dev|844|765|391|2,000|
|Dev Test|994|681|325|2,000|
|Test|11,603|15,269|5,137|30,009|

Датотеките кои се преземаат од официјалната страница натпреварот SemEval соддржат само id на секој од твитовите кои треба да се користат, не и самиот текст (освен тест множеството). Се користи скрипта од [2] за да се преземе текстот преку Twitter API, но за време на изработката на проектот, некои твитови не беа достапни (најверојатно избришани од самите корисници). 

||Позитивни|Неутрални|Негативни|*Вкупно*|
|----------|--------:|--------:|-------:|
|Train|2,569|1,684|701|4,954|
|Dev|708|639|327|1,674|
|Dev Test|822|584|259|1,998|
|Test|11,603|15,269|5,137|30,009|

Датотеките кои ќе ги користиме директно како влез во нашите модели се наоѓаат во директориумот `tweet_input`
Имињата на датотеките кои ќе ги користиме и нивната структура е следнава:

* `train_and_dev_data/tweet_input/train_input.tsv`
  со формат tweet_text\tweet_class каде tweet_text текст на твитот, \t е таб делимитер, и tweet_class е класата на која припаѓа твитот, positive, negative или neutral
* `train_and_dev_data/tweet_input/test_input.tsv` со формат x\ttweet_text, каде x e реден број на твитот, \t е таб делимитер, и tweet_text е текст на твитот 
  

## Non-deep-learning Bag of Words модел

Еден од моделите кои ќе ги градиме се базира на пристапот Bag-of-Words. Сите твитови кои ги имаме за тренирање, по нивното чистење кое ќе го опишеме подолу ќе ги искористиме за да изградиме речник на најчести зборови. Од тестирањето, според големината на тренинг множеството (~5000 твитови) не добиваме зголемување на точноста за речник со повеќе од 5000 зборови. Откако речникот ќе биде генериран, зборовите во него се подредуваат, потоа секој од твитовите се претставува во форма на вектор. Доколку во речникот има n зборови, секој од твитовите се трансформира во вектор со n членови, на секоја позиција се става бројот колку пати се појавува одреден збор од речникот во текстот.

Пример, нека речникот ги содржи зборовите: [една, јаболка, портокал, мајка, татко].
Текстот „мајка ми изеде една јаболка“ би се трансформирал во векторот (1,1,0,1,0), а текстот „портокал, портокал, сакам портокал“ би бил (0,0,3,0,0). За креирање на речникот на најчести зборови и трансформација на твитовите го користевме пакетот sklearn во Python, или поконретно CountVectorizer

Вака добиените вектори за твитовите заедно со класите на кои припаѓа секој од нив (positive, negative, neutral) се предаваат на класификатор на тренирање со што се добива истрениран модел. Класификатори кои ќе ги испробаме се: `RandomForestClassifier`, `SVM`, `LogisticRegression`, `ExtraTreesClassifier`. Сите имплементации ги користевме од `sklearn` пакетот. 

### Претпроцесирање

Пред да започнеме со градењето на модел ќе треба податочното множество да се прочисти со цел да останат само деловите од текстот кои можат да имат најголемо семантичко значење.  

Ќе започнеме со дефинирање некои константи кои ќе се користат во понатамошното извршување

In [1]:
# will hold the training set file lines
input_lines = []

# training data set, each line = (tweet, class)
train_file_name = "train_and_dev_data/tweet_input/train_input.tsv"
print("Reading training data set from: ", train_file_name)

# test data set, each line = (index/ line no., tweet)
test_file_name = "train_and_dev_data/tweet_input/test_input.tsv"
print("Reading test data set from: ", test_file_name)

# solutions file, each line = (index, correct class)
solutions_file_name = "train_and_dev_data/tweet_output/test_solutions.tsv"
print("Solutions read from: ", solutions_file_name)

# flags to test different variations of pre-processing
remove_stopwords = False
retain_letters_only = True
print('-----')
print("Will stopwords be removed?:", remove_stopwords)
print("Will only letters be left in the tweet text?: ", retain_letters_only)

Reading training data set from:  train_and_dev_data/tweet_input/train_input.tsv
Reading test data set from:  train_and_dev_data/tweet_input/test_input.tsv
Solutions read from:  train_and_dev_data/tweet_output/test_solutions.tsv
-----
Will stopwords be removed?: False
Will only letters be left in the tweet text?:  True


In [2]:
# read all the training tweets in a list
with open(train_file_name) as f:
    input_lines = f.readlines()
print("Read ", len(input_lines), " from training set file")

Read  4954  from training set file


Во листата `input_lines` се содржат линиите од датотеката што го претставуваше тренинг множеството. Првото нешто што го правиме е ги трансформираме линиите во подреден пар (tuple) од 2 листи, во првата листа ќе се наоѓаат текстот на твитовите, а во втората листа класите на секој од твитовите, таа трансформација ќе ја направиме со функцијата `get_tuple_from_input_file`

In [3]:
def get_tuple_from_input_file(lines_with_tweet_and_class, delimiter):
    """splits each string line tweet\tclass and appends them to tweets[] and classes[] accordingly and 
        returns them as (tweets, classes)"""
    tweets = []
    classes = []
    for tweet in lines_with_tweet_and_class:
        splits = tweet.split(delimiter)
        tweets.append(splits[0])
        classes.append(splits[1])

    return tweets, classes


In [4]:
# convert from "tweet\tclass"[] to (tweet[], class[])
tweets_and_class_tuple = get_tuple_from_input_file(input_lines, "\t")
print("Number of tweets: ", len(tweets_and_class_tuple[0]))
print("Number of classes ", len(tweets_and_class_tuple[1]))

Number of tweets:  4954
Number of classes  4954


### Отстранување на линкови

Линковите кои се споделени во твитовите не соддржат никакви информации, уште повеќе што твитер ги крати линковите за да скрати на карактери. Нивното отстранување може само да ни помогне. За отстранување на линковите го користевме регуларниот израз (https?://\S+) бидејќи во твитовите сите беа означени со http(s) протоколот на почеток.


In [5]:
import re

def remove_pattern_from_string(given_string, re_compiled_pattern):
    """removes a re compiled regex pattern from a given string """
    return re_compiled_pattern.sub("", given_string)

def remove_links_from_tweets(tweets_list):
    """for a given list of tweet texts removes all links starting with http:// or https://"""
    regex = re.compile("(https?://\S+)", re.IGNORECASE)
    tweets_without_links = []
    for tweet in tweets_list:
        clean_tweet = remove_pattern_from_string(tweet, regex)
        tweets_without_links.append(clean_tweet)
    return tweets_without_links

#let's test it out
example_tweets = ['i like pie', 'if you like go to http://pie.com now']
example_tweets = remove_links_from_tweets(example_tweets)
print("after removing links: ", example_tweets)

after removing links:  ['i like pie', 'if you like go to  now']


### Процесирање на хаштази (hashtags)
Хаштагот може да го гледаме како начин на означување дека твитот се однесува на одредена тема. Хаштаг може да биде кратенка (#FIFA), регуларен збор од англискиот речник (#love), спој од повеќе зборови од англискиот речник (#OhMyGod). Начини да се процесираат хаштазите се: да се отстранат, да се избрише тараба знакот # и да се третираат како и останатите зборови, да се остават како што се. Отстранувањето на хаштаг знакот и третирањето како обични зборови сметаме дека нема да даде добри резултати затоа што најчесто хаштазите не се обични зборови. 

### Отстранување на броеви и специјални знаци
Последна трансформација што ќе ја направиме на текстот е да ги отстраниме сите карактери кои се броеви или специјални знаци, да ги оставиме само буквите. Бидејќи пристапот кој го одбравме се базира на градење на речник на најчести зборови, оваа трансформација ни е потребна за да ни останат само зборови кои би го составиле овој речник. Но хаштаг зборовите ќе ги оставиме непроменети.

Овие две претпроцесирања ќе ги правиме со една функција

In [6]:
def tweet_to_list_of_words(tweet, retain_letters_only=True, remove_hashsign=False):
    """cleans a tweet but returns the tweet as a list of words. 'hello 34 apple' = ['hello', 'apple'] """
    letters_only_regex = re.compile("[^a-zA-Z]")
    words = tweet.lower().split()
    words_to_return = []
    for word in words:
        if word.startswith("#"):
            if remove_hashsign:
                word = word[1:]
            if len(word) > 0:
                words_to_return.append(word)
        elif retain_letters_only:
            replaced = letters_only_regex.sub("", word)
            if len(replaced) > 0 and replaced != "th":
                words_to_return.append(replaced)
        else:
            words_to_return.append(word)

    return words_to_return

example_tweets = ['i like pie', 'if you like go to http://pie.com now']
example_tweets = remove_links_from_tweets(example_tweets)
example_tweets = [tweet_to_list_of_words(twt, True, False) for twt in example_tweets]
print(example_tweets)

[['i', 'like', 'pie'], ['if', 'you', 'like', 'go', 'to', 'now']]


Претпроцесирањето во чисти зборови ќе го правиме со следната функција која прво ги оттргнува линковите од твитовите, па ги трга стоп-збоовите (ако е повикана со соодветен аргумент за тоа), ги чисти броевите и специјалните симболи (ако е сетирано во аргументите тоа да се прави) и враќа листа од чисти твитови

In [7]:
import nltk.data
from nltk.corpus import stopwords

def pre_process_tweets_bag_of_words(list_of_tweets, 
                                    remove_stopwords=False, 
                                    retain_letters_only=True,
                                    remove_hashsign=False):
    """takes a list of tweets, cleans them (removes stopwords, leaves only letters, removes # from hashtags)
    returns list of clean tweets (tweet[] => clean_tweet[])"""
    tweets_with_no_links = remove_links_from_tweets(list_of_tweets)
    english_stopwords = set(stopwords.words("english"))
    preprocessed = []
    for tweet in tweets_with_no_links:
        tweet_words = tweet_to_list_of_words(tweet, retain_letters_only, remove_hashsign)
        if remove_stopwords:
            meaningful_words = remove_from_list_of_words(tweet_words, english_stopwords)
            preprocessed.append(" ".join(meaningful_words))
        else:
            preprocessed.append(" ".join(tweet_words))

    return preprocessed


Сега да се навратиме на листата од твитови која ја прочитавме од датотеката за тренирање, и листата од твитови ја чистиме

In [8]:
# tweets are cleaned (links removed, stopwords, numbers and special characters)
# returns a list of clean tweets, but each tweet is represented as a string not a list of words
clean_tweet_words = pre_process_tweets_bag_of_words(tweets_and_class_tuple[0],
                                                    remove_stopwords=remove_stopwords,
                                                    retain_letters_only=retain_letters_only)
print("Number of clean tweets: ", len(clean_tweet_words))
print("Example clean tweets: ")
for tweet in clean_tweet_words[0:5]:
    print('\t', tweet)

Number of clean tweets:  4954
Example clean tweets: 
	 we think its much better than kids who obsess over people like jayz amp miley cyrus
	 i didnt think this was possible but my apple watch has battery left right now amp i last took it off the charger tuesday around am
	 bernie sanders will stop in greensboro nc sunday hes holding a rally at the greensboro coliseum #vote2016
	 lets see how many couples are going to start dry humping when eric church sings wrecking ball tomorrow #countryjam
	 oh shit might be going to ihop tomorrow elliott gone kill me


Од чистите твитови ќе изградиме речник, користејќи го CountVectorizer од sklearn пакетот

### Градење на модел - Non Deep Learning Bag of Words
Еден од моделите кои ги изградивме се базира на пристапот Bag-of-Words. Сите твитови кои ги имаме за тренирање, по нивното чистење опишано во секција 3, ги искористивме за да изградиме речник на најчести зборови. Од тестирањето, според големината на тренинг множеството (~5000 твитови) не добивме зголемување на точноста за речник со повеќе од 5000 зборови. Откако речникот е генериран, зборовите во него се подредуваат, потоа секој од твитовите се претставува во форма на вектор. Доколку во речникот има n зборови, секој од твитовите се трансформира во вектор со n членови, на секоја позиција се става бројот колку пати се појавува одреден збор од речникот во текстот.
Пример, нека речникот ги содржи зборовите: [една, јаболка, портокал, мајка, татко].
Текстот „мајка ми изеде една јаболка“ би се трансформирал во векторот (1,1,0,1,0), а текстот „портокал, портокал, сакам портокал“ би бил (0,0,3,0,0). За креирање на речникот на најчести зборови и трансформација на твитовите го користевме пакетот sklearn во Python, или поконретно CountVectorizer

Вака добиените вектори за твитовите заедно со класите на кои припаѓа секој од нив (positive, negative, neutral) се предаваат на класификатор на тренирање со што се добива истрениран модел. Класификатори кои ги испробавме се: RandomForestClassifier, SVM, LogisticRegression, ExtraTreesClassifier. Сите имплементации ги користевме од sklearn пакетот. 


In [9]:
from sklearn.feature_extraction.text import CountVectorizer

print("Creating the bag of words ...")
vectorizer = CountVectorizer(analyzer="word",
                             tokenizer=None,
                             preprocessor=None,
                             stop_words=None,
                             max_features=5000)

# each tweet is converted to a vector using the bag of words method
train_data_features = vectorizer.fit_transform(clean_tweet_words)
train_data_features = train_data_features.toarray()
vocab = vectorizer.get_feature_names()

print("Vocabulary is complete. ")
print("Example words: ")
for word in vocab[1:10]:
    print(word)

Creating the bag of words ...
Vocabulary is complete. 
Example words: 
aapl
abc
abigail
able
about
above
absence
absolute
absolutely


In [10]:
import sklearn.svm as svm
# from sklearn.linear_model import LogisticRegression
# from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier

classifier = RandomForestClassifier(n_estimators=200)
# classifier = ExtraTreesClassifier(n_estimators=200, n_jobs=-1, verbose=False,
#                                   class_weight='auto',
#                                   bootstrap=True)
# classifier = LogisticRegression(verbose=False)
print("Training the classifier, if svm will take a while")
# max_iterations = 10 #for dev purposes, set -1 for infinity
classifier = classifier.fit(train_data_features, tweets_and_class_tuple[1])
print("Classifier finished training")

Training the classifier, if svm will take a while
Classifier finished training


Откако ќе се истренира класификаторот, треба да ги вчитаме податоците од тест множеството, и нив како и тренинг множеството ги обработуваме. Прво мора да дефинираме една нова функција која од тест множеството ќе направи торка од твитовите и нивните редни броеви

In [11]:
def get_tuple_from_test_input_file(tweets_with_number, delimiter):
    """splits each tuple (index, tweet) adding the results in into tweets and indexes returns (tweets, indexes)"""
    tweets = []
    index_numbers = []
    for tweet in tweets_with_number:
        splits = tweet.split(delimiter)
        tweets.append(splits[1])
        index_numbers.append(splits[0])

    return tweets, index_numbers

Ја читаме датотеката со тест множеството, ги чистиме твитовите со истите поставки како и тренинг множеството

In [12]:
test_lines = []
with open(test_file_name) as f:
    test_lines = f.readlines()

print("Cleaning test tweets")
# convert from (index, tweet)[] to (index[], tweet[])
test_tweets_and_index = get_tuple_from_test_input_file(test_lines, "\t")
clean_test_tweet_words = pre_process_tweets_bag_of_words(test_tweets_and_index[0],
                                                         remove_stopwords=remove_stopwords,
                                                         retain_letters_only=retain_letters_only)
print("Number of clean tweets: ", len(clean_test_tweet_words))

Cleaning test tweets
Number of clean tweets:  32009


Потоа со постоечкиот речник ги претвараме тест твитовите во торба од зборови и го користиме класификаторот што претходно го тренивравме за да направивме предвидувања

In [13]:
# create a vector representation of the test tweets
test_data_features = vectorizer.transform(clean_test_tweet_words)
test_data_features = test_data_features.toarray()

print("Starting prediction")
result = classifier.predict(test_data_features)
print("Predicition done")

Starting prediction
Predicition done


Следно ја читаме датотеката со решенијата

In [14]:
solutions = []
with open(solutions_file_name) as f:
    solutions = f.readlines()

solution_and_index = get_tuple_from_test_input_file(solutions, "\t")
solutions = solution_and_index[0]

За одредување на перформансите, (точноста) на класифицирањето ќе ја користиме истата мерка дефинирана во SemEval натпреаварот[1]. 
Мерката се дефинира како:
![f1](f1_pn.png "F1_pn")
F<sub>1</sub><sup>P</sup> е оцената за класата POSITIVE:
![f1_p](f1_p.png "F1_p")
Овде $\pi$<sup>P</sup> и $\rho$<sup>P</sup> ги означуваат precision и recall за POSITIVE класата, или:
![pi_p](pi_p.png "Pi_p")
![ro_p](ro_p.png "Ro_p")
Каде PP, UP, NP, PU, PN се ќелиите во следнава конфузиона матрица:

|            |          |          |Вистинска|          |
|-----------:|---------:|---------:|--------:|---------:|
|            |          | POSITIVE | NEUTRAL | NEGATIVE |
|            | POSITIVE |    PP    |    PU   |    PN    |
| Предвидена | NEUTRAL  |    UP    |    UU   |    UN    |
|            | NEGATIVE |    NP    |    NU   |    NN    |

Ќелија XY oд конфузионата матрица означува „број на твитови што класификаторот ги препознал како X, а вистинската класа била Y“. Аналогно на F<sub>1</sub><sup>P</sup> се дефинира и F<sub>1</sub><sup>N</sup>. 
За споредба ќе ги користиме резултатите од SemEval2016, за истата задача дадени во [1] каде најдобриот тим има добиено резултат F<sub>1</sub><sup>PN</sup> = 0.633, а најслабиот тим е со резултат F<sub>1</sub><sup>PN</sup> = 0.303


In [17]:
POSITIVE = "positive"
NEGATIVE = "negative"
NEUTRAL = "neutral"


def calculate_score(solutions, result):
    if len(solutions) != len(result):
        raise Exception(
            "solutions and result are not the same length, " + str(len(solutions)) + " vs " + str(len(result)))

    confusion_matrix = {}

    total = len(solutions)
    for i in range(0, total):
        result_class = result[i].strip()
        if result_class not in confusion_matrix:
            confusion_matrix[result_class] = {}

        result_row = confusion_matrix[result_class]
        solution_class = solutions[i].strip()
        if solution_class not in result_row:
            result_row[solution_class] = 0.0

        result_row[solution_class] += 1.0

    PP = confusion_matrix[POSITIVE][POSITIVE]
    PU = confusion_matrix[POSITIVE][NEUTRAL]
    PN = confusion_matrix[POSITIVE][NEGATIVE]
    UP = confusion_matrix[NEUTRAL][POSITIVE]
    NP = confusion_matrix[NEGATIVE][POSITIVE]
    NN = confusion_matrix[NEGATIVE][NEGATIVE]
    NU = confusion_matrix[NEGATIVE][NEUTRAL]
    UN = confusion_matrix[NEUTRAL][NEGATIVE]

    precision_p = PP / (PP + PU + PN)
    recall_p = PP / (PP + UP + NP)
    F1_P = (2 * precision_p * recall_p) / (precision_p + recall_p)

    precision_n = NN / (NP + NU + PN)
    recall_n = NN / (NN + UN + PN)
    F1_N = (2 * precision_n * recall_n) / (precision_n + recall_n)
    return (F1_P + F1_N) / 2


In [18]:
f1 = calculate_score(solutions, result)
print("Calculated F1 score: ", f1)

KeyError: 'positive'

### Резултати од Non Deel Learning - Bag of Words

|       Класификатор      |  Стоп-зборови |F<sub>1</sub><sup>PN</sup> |
|------------------------:|--------------:|--------------------------:|
|      Random Forest      |  отстранети   |          0.2871           |
|      Random Forest      | неотстранети  |          0.2786           |
| Extra Trees Classifier  |  отстранети   |          0.297            |
| Extra Trees Classifier  | неотстранети  |          0.2966           |
|   Logistic Regression   |  отстранети   |          0.4024           |
|   Logistic Regression   | неотстранети  |          0.3852           |
|           SVM           |  отстранети   |          0.4293           |
|           SVM           | неотстранети  |          0.5494           |

### Градење на модел - Deep Learning - Bag of Centroids

Тим од истражувачи од Google предводени од Томас Миколов имаат развиено архитектура за креирање на векторска репрезентација на зборови од големи множества на реченици. Со нивната архитектура се добиваат модели кои успеваат семантички и синтактички слични зборови да ги престават со просторно блиски вектори [5, 6]. Постои библиотека која претставува имплементација на оваа архитектура, која за Python може да се најде во пакетот gensim [7,8].

Ќе ја искористиме идејата дадена во Word2Vec NLP Tutorial на Kaggle[9]. 
Прво потребно е да се изгради моделот на Word2Vec. Се обидовме да изградиме Word2Vec модел со твитовите кои ги имаме на располагање, но тие беа мало множество и моделот не покажуваше добри карактеристики (не наоѓаше никаква семантичка сличност). Затоа зедовме веќе изграден модел кој се користи во туторијалот на Kaggle. Моделот таму е изграден со користење на податочно множество на коментари и рецензии на филмови на англиски јазик. 

Бидејќи зборовите во моделот во суштина се само вектори, можеме едноставно да ги кластерираме. Го користиме k-means за да ги добиеме кластерите, притоа одбираме голем број на кластери со релативно малку зборови во кластерите. `Број на кластери = Број на зборови во моделот / 5`. Вака добиените кластери ги користиме за да креираме торба од центроиди. За секој од твитовите наоѓаме секој од зборовите во кој од креираните кластери би припаѓал. И добиваме векторска репрезентација за секој твит. Понатаму со оваа репрезентација на твитовите можеме да тренираме било каков класификатор.

Направивме две варијанти во претпроцесирањето така што од хаштазите ги тргавме # карактерите, бидејќи ниеден хаштаг не е претставен во Word2Vec моделот, но сакавме таму каде хаштазите се вистински зборови да ја искористиме нивната информација. 

Како и за претходниот модел ги специфицираме знаменцата кои ги дефинираат различните начини на претпроцесирање, и кои се тест и тренинг датотеките. Воедно го вчитуваме и Word2Vec моделот, потоа ги прочистуваме твитовите како и претходно. Мала варијација постои во начинот кој е потребен да се спремаат твитовите

In [19]:
def tweets_to_wordlist(tweets, remove_stopwords=False, retain_letters_only=True, remove_tarabas=False):
    """takes a list of tweets, cleans each tweet, but returns a list of tweetwords
    tweets[]=> (words[])[]"""
    tweets_with_no_links = remove_links_from_tweets(tweets)
    preprocessed = []
    english_stopwords = set(stopwords.words("english"))
    for tweet in tweets_with_no_links:
        tweet_wordlist = tweet_to_list_of_words(tweet, retain_letters_only, remove_tarabas)
        if remove_stopwords:
            meaningful_words = remove_from_list_of_words(tweet_wordlist, english_stopwords)
            preprocessed.append(meaningful_words)
        else:
            preprocessed.append(tweet_wordlist)

    return preprocessed


In [20]:
from gensim.models import Word2Vec

input_lines = []
file_name = "train_and_dev_data/tweet_input/train_input.tsv"
test_file_name = "train_and_dev_data/tweet_input/test_input.tsv"
solutions_file_name = "train_and_dev_data/tweet_output/test_solutions.tsv"
remove_stopwords = False
retain_letters_only = True
remove_tarabas = True

# word2vecModel = Word2Vec.load("train_and_dev_data/word2vec_model/300features_40minwords_10context")

print("Reading input from: ", file_name)
with open(file_name) as f:
    input_lines = f.readlines()

tweets_and_class_tuple = get_tuple_from_input_file(input_lines, "\t")

print("Cleaning the training tweets")
clean_tweets = tweets_to_wordlist(
    tweets_and_class_tuple[0],
    remove_stopwords=remove_stopwords,
    retain_letters_only=retain_letters_only,
    remove_tarabas=remove_tarabas)
print("Training tweets are clean")

test_lines = []
with open(test_file_name) as f:
    test_lines = f.readlines()

print("Cleaning test tweets")
test_tweets_and_index = get_tuple_from_test_input_file(test_lines, "\t")
clean_test_tweets = tweets_to_wordlist(
    test_tweets_and_index[0],
    remove_stopwords=remove_stopwords,
    retain_letters_only=retain_letters_only,
    remove_tarabas=remove_tarabas)

print("Test tweets are clean")

Reading input from:  train_and_dev_data/tweet_input/train_input.tsv
Cleaning the training tweets
Training tweets are clean
Cleaning test tweets
Test tweets are clean


Кластерирањето ќе го направиме со k-means

In [21]:
# word_vectors = word2vecModel.syn0
# num_clusters = int(word_vectors.shape[0] / 5)

# from sklearn.cluster import KMeans

#verbose output is on, number of jobs = as many cores available
# kmeans_clustering = KMeans(n_clusters=num_clusters, verbose=1, n_jobs=-1)
# print("Starting KMeans")
# idx = kmeans_clustering.fit_predict(word_vectors)
# print("KMeans done")

Откако се готови кластерите, потребно е секој од тренинг и тест твитовите да ги претставиме како bag-of-centroids 

In [22]:
def create_bag_of_centroids(wordlist, word_centroid_map):
    #
    # The number of clusters is equal to the highest cluster index
    # in the word / centroid map
    num_centroids = max(word_centroid_map.values()) + 1
    #
    # Pre-allocate the bag of centroids vector (for speed)
    bag_of_centroids = np.zeros(num_centroids, dtype="float32")
    #
    # Loop over the words in the review. If the word is in the vocabulary,
    # find which cluster it belongs to, and increment that cluster count
    # by one
    for word in wordlist:
        if word in word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1
    #
    # Return the "bag of centroids"
    return bag_of_centroids

In [27]:
# word_centroid_map = dict(zip(word2vecModel.index2word, idx))
import pickle
import numpy as np

idx = pickle.load(open("train_and_dev_data/idx", "rb"))
index2word = pickle.load(open("train_and_dev_data/index2word", "rb"))
word_centroid_map = dict(zip(index2word, idx))
num_clusters = num_centroids = max(word_centroid_map.values()) + 1
train_centroids = np.zeros((len(clean_tweets), num_clusters),
                           dtype="float32")

# transform the training set tweets into bags of centroids
counter = 0
for tweet in clean_tweets:
    train_centroids[counter] = create_bag_of_centroids(tweet, word_centroid_map)
    counter += 1

# repeat for test tweets
test_centroids = np.zeros((len(clean_test_tweets), num_clusters), dtype="float32")

counter = 0
for tweet in clean_test_tweets:
    test_centroids[counter] = create_bag_of_centroids(tweet, word_centroid_map)
    counter += 1

Потоа тренираме класификатор со тренинг твитовите, овој пат претставени преку центроидите, и потоа го тестираме класификаторот

In [28]:
# fit a svm and extract predictions
print("Training the classifier")
classifier = svm.SVC(kernel='rbf', C=10, gamma=0.001, max_iter=-1,
                     verbose=False, class_weight='balanced', cache_size=4000,
                     probability=True)
classifier = classifier.fit(train_centroids, tweets_and_class_tuple[1])
print("Training done")

print("Starting prediction")
result = classifier.predict(test_centroids)

print("Prediction done, starting evaluation")
solutions = []
with open(solutions_file_name) as f:
    solutions = f.readlines()

solution_and_index = get_tuple_from_test_input_file(solutions, "\t")
solutions = solution_and_index[0]
precision = calculate_score(solutions, result)
print("Precision: ",precision)

Training the classifier
Training done
Starting prediction
Prediction done, starting evaluation
Precision:  0.4678386343076686


Добивањето на резултати со овој метод трае значително многу затоа што процесот на кластерирање на зборовите ов Word2Vec моделот е доста бавен, (неколку саати). Може да се забрза доколку се зачува/серијализира структурата на кластерите. 
Откако ги добивме репрезентациите на твитовите во векторска форма така што секој од зборовите беше означен на кој кластер припаѓа искористивме SVM класификатор. Стоп зборовите не беа отстранети, само буквите беа заддржани и линковите отстранети. 

Без отстранување на тараба карактерите од хаштазите се доби резултат F<sub>1</sub><sup>PN</sup> = 0.474, а ако се отстранат тогаш се добива F<sub>1</sub><sup>PN</sup> = 0.467.

## Референци

1. Nakov, Preslav, et al. "SemEval-2016 task 4: Sentiment analysis in Twitter." Proceedings of SemEval (2016): 1-18. 
2. Semeval Twitter data download script, https://github.com/aritter/twitter_download
3. SemEval-2016 Task 4 test datasets, zip, 3.4 MB, http://alt.qcri.org/semeval2016/task4/data/uploads/semeval2016_task4_test_datasets.zip
4. SemEval-2016 Task 4 Train, Dev and DevTest data, zip, 6.8 MB http://alt.qcri.org/semeval2016/task4/data/uploads/semeval2016-task4.traindevdevtest.v1.2.zip
5. Mikolov, Tomas, et al. "Efficient estimation of word representations in vector space." arXiv preprint arXiv:1301.3781 (2013).
6. Mikolov, Tomas, et al. "Distributed representations of words and phrases and their compositionality." Advances in neural information processing systems. 2013.
7. Gensim, topic modeling for humans, website: https://radimrehurek.com/gensim/
8. Python Framework for fast vector space modeling, PyPi package https://pypi.python.org/pypi/gensim
9. https://www.kaggle.com/c/word2vec-nlp-tutorial
